# <center>Project 4 </center>
## <center>Stefan Popov</center><br>
 <center>Nov 25th 2016</center><br><br>
<center>MATH3316/CSE3366</center><br><br>







### 1. High order numerical integration
In the first part of this project, we were to create our own composite numerical integration function,

In [ ]:
double composite_int(Fcn& f, const double a, const double b
                     const int n);

in a file composite_int.cpp. The function was to implement a composite numerical integration formula that is $O(h^8)$ accurate of our own choosing. For the purposes of this assignment, I chose to implement the 4 point Gaussian quadrature rule. The function implementation was as follows : 

In [ ]:
double composite_int(Fcn & f, const double a,const double b,const int n){
	//get midpoint
	double h = (b-a)/n;

	//set weights
	double x1 = -0.86113631159405;
	double x2 = -0.33998104358486;
	double x3 = 0.33998104358486;
	double x4 = 0.86113631159405;
	double w1 = 0.34785484513745;
	double w2 = 0.65214515486254;
	double w3 = 0.65214515486255;
	double w4 = 0.34785484513745;

	double result=0.0;

	double xmid,node1, node2,node3,node4;

	//iterate as many times as needed 
	for(int i=0 ;i<n;i++){
		//calculate result at each node
		xmid= a+ (i+0.5)*h;
		node1= xmid+ 0.5*h*x1;
		node2= xmid+ 0.5*h*x2;
		node3= xmid+ 0.5*h*x3;
		node4= xmid+ 0.5*h*x4;
		//update result
		result+=w1*f(node1)+w2*f(node2)+w3*f(node3) + w4*f(node4);
	}

	return (0.5*h*result);
}

Then we were to create a C++ main() routine in a file named test_int.cpp that would emulate the already provided routine in test_Gauss2.cpp to ensure that the integration routine works as it is supposed. It is to use the same integrand as the supplied routine, but modified to where it uses a different values of n to demonstrate the convergence rate of the method, where we should use at least 6 n values. The main() routine was as follows : 

In [ ]:
int main(){

	double a = -3.0;
	double b =5.0;
	fcn f;
	f.c = 0.5;
	f.d=25.0;
	double Itrue = f.antiderivative(b) - f.antiderivative(a);
	printf("\n True Integral = %22.16e\n", Itrue);


	// test the Gauss-4 rule	
	cout << "\n Gauss-4 approximation:\n";
	cout << "     n             R(f)            relerr    conv rate\n";
	cout << "  ---------------------------------------------------\n";
	vector<int> n = {20, 40, 80, 160, 320, 640, 1280, 2560, 5120, 10240};
	vector<double> errors(n.size());
	vector<double> hvals(n.size());

	// iterate over n values, computing approximations, error, convergence rate
	double Iapprox;
	for (int i=0; i<n.size(); i++) {

		printf("   %6i", n[i]);
		Iapprox = composite_int(f, a, b, n[i]);
		errors[i] = fabs(Itrue-Iapprox)/fabs(Itrue);
		hvals[i] = (b-a)/n[i];
		if (i == 0) 
			printf("  %22.16e  %7.1e     ----\n", Iapprox, errors[i]);
		else 
		printf("  %22.16e  %7.1e   %f\n", Iapprox, errors[i], 
			(log(errors[i-1]) - log(errors[i]))/(log(hvals[i-1]) - log(hvals[i])));
	}
	cout << "  ---------------------------------------------------\n";
}

When ran, it yielded the following output : 

### 2. Adaptive Numerical Integration

In this part of the project, we were to create an adaptive numerical integration function, with the following signature :

In [ ]:
int adaptive_int(Fcn& f, const double a, const double b, 
                const double rtol, const double atol,
                double& R, int& n, int& Ntot);

in a new file called adaptive_int.cpp that will compute the integral<br><br> $$\int_{a}^{b} f(x)dx$$ <br><br> 
by calling the composite_int() function from the previous section in an adaptive manner. The main goal was to construct an approximation of the true integral such that $$ |I(f) - R_n(f)| < rtol|I(f)| + atol,$$
where $$I(f) = \int_{a}^{b} f(x)dx$$ and $R_n(f)$=composite_int(f,a,b,n) such that the achieved result takes the least possible computational effort and n is minized while doing so. <br>
The challenge with the error bound would be the fact that we do not know the value of the true integral I(f), but for rapidly-convergent methods, an approximation using more subintervals would be more accurate than $R_n(f)$. Hence the equation will be modeled as $$ |R_{n+k}(f) - R_n(f)|< rtol|R{n+k}(f)| + atol.$$ The n here is to be adaptively updated until the bound is satisfied in the least computational effort. <br><br> Upon completion, the adaptive_int() function should fill R with its double-precision approximation of the integral, n with the final number of intervals it used in computing R. Ntot would hold the total number of intervals used along the way and it's return value should be 0 if it believes that it successfully satisfied the approximation of the error bound or 1 if it failed. <br><br> This was all achieved in the following code